In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow --quiet

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from keras.callbacks import ModelCheckpoint

In [6]:
#Loading the dataset
df = pd.read_csv("PLCdata.csv",index_col=[0])
#combining the fault, model and PLC columns into 1 column fault_model
df['Fault_Model'] = df['Fault'] + " " + df['Model']+ " " + df["PLC"]
df['Diagnostic_Corrective Action'] = df['Diagnostic'] + " " + df['Corrective Action']
df.drop(['Fault','Model','PLC'], inplace=True, axis=1)

In [7]:
df.head()

,Diagnostic,Corrective Action,Fault_Model,Diagnostic_Corrective Action
0,broken or cracked glass electrode,replace electrode if cracked,cracked glass failure model e emerson,broken or cracked glass electrode replace elec...
1,broken or cracked glass electrode,check wiring for short,cracked glass failure model e emerson,broken or cracked glass electrode check wiring...
2,reference electrode poisoned,replace reference electorode,zero offset error model e emerson,reference electrode poisoned replace reference...
3,coated reference electrode,clean electrode as instructed in sensor manual,high reference imped model e emerson,coated reference electrode clean electrode as ...
4,sensor out of process,clean electrode as instructed in sensor manual,high reference imped model e emerson,sensor out of process clean electrode as instr...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1398 entries, 0 to 1397
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Diagnostic                    1398 non-null   object
 1   Corrective Action             1397 non-null   object
 2   Fault_Model                   1398 non-null   object
 3   Diagnostic_Corrective Action  1397 non-null   object
dtypes: object(4)
memory usage: 54.6+ KB


In [11]:


data = df

# Convert the title_summaries column to integers using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(data['Diagnostic_Corrective Action'])
encoded_labels = label_encoder.transform(data['Diagnostic_Corrective Action'])

# Tokenize the text and query columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Diagnostic_Corrective Action'].tolist() + data['Fault_Model'].tolist())

# Get the maximum sequence length
max_sequence_length = max([len(x.split()) for x in data['Diagnostic_Corrective Action'].tolist() + data['Fault_Model'].tolist()])

# Convert the text and query columns to sequences
text_sequences = tokenizer.texts_to_sequences(data['Diagnostic_Corrective Action'].tolist())
text_sequences_padded = pad_sequences(text_sequences, maxlen=max_sequence_length)
query_sequences = tokenizer.texts_to_sequences(data['Fault_Model'].tolist())
query_sequences_padded = pad_sequences(query_sequences, maxlen=max_sequence_length)

# Define the LSTM model architecture
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_sequence_length)(input_layer)
lstm_layer = LSTM(128)(embedding_layer)
output_layer = Dense(len(data['Diagnostic_Corrective Action'].unique()), activation='softmax')(lstm_layer)
model = Model(inputs=input_layer, outputs=output_layer)


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


checkpoint = ModelCheckpoint('best_model.hdf5', monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

# Train the model with one-hot encoded labels
model.fit(text_sequences_padded, tf.keras.utils.to_categorical(encoded_labels), epochs=20,callbacks=callbacks_list)
model.save('final_model.h5')


In [ ]:
!pip install streamlit --quiet
! pip install pyngrok --quiet

In [ ]:
%%writefile app.py
import streamlit as st
st.write("# PLC Fault Prediction")
st.text_input("Enter the PLC:")
st.text_input("Enter the Model:")
st.text_input("Enter the paper Fault:")

Overwriting app.py


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2LArVCZnMmuPMV0uYjEx4zAOSpx_6AiMAuvnRpVMQxFzSkvaE") 

!nohup streamlit run app.py --server.port 80 & --quiet
url = ngrok.connect(port = '80')
print(url)

/bin/bash: --quiet: command not found
nohup: appending output to 'nohup.out'
NgrokTunnel: "http://6b6c-34-83-144-170.ngrok.io" -> "http://localhost:80"


In [ ]:
ngrok.kill()